In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from dataLoader import importData
import tensorflow as tf
import numpy as np
x_train, y_train, x_test, y_test, _ = importData(5)

Data-set: mnist


In [2]:
from soft_max_classifier import get_train_val_subsets
import os

def train_model(x_train, y_train, num_epochs=20, batch_size=64, 
                checkpoint_filepath = 'checkpoint/', verbose=True):
    x_train_, y_train_, x_val_, y_val_ = get_train_val_subsets(x_train, y_train)
    nClass = len(np.unique(y_train))
    num_dim = x_train.shape[1]

    model = tf.keras.models.Sequential([
    
    tf.keras.layers.InputLayer(input_shape=(num_dim,)),
    tf.keras.layers.Dense(1000, activation = 'relu'),
    tf.keras.layers.Dense(1000, activation = 'relu'),
    #tf.keras.layers.Dense(1000, activation = 'linear'),
    tf.keras.layers.Dense(nClass, activation = 'softmax')
    ])


    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=['accuracy'],
    )

    if not os.path.exists(checkpoint_filepath):
        os.makedirs(checkpoint_filepath)

    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        #filepath= checkpoint_filepath + '{epoch:02d}-{val_loss:.4f}.hdf5',
        filepath = checkpoint_filepath+'best_val_acc.hdf5',
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        metric = 'val_accuracy',
        save_freq = 'epoch', verbose = verbose)

    model.fit(x_train_, y_train_, epochs=num_epochs, validation_data=(x_val_, y_val_), 
        batch_size = batch_size, callbacks=[model_checkpoint_callback])

    model.load_weights(checkpoint_filepath + 'best_val_acc.hdf5')
    return model

In [3]:
model = train_model(x_train, y_train)

class labels  [0 1 2 3 4 5 6 7 8 9]
Epoch 1/20
827/844 [============================>.] - ETA: 0s - loss: 4.9915 - accuracy: 0.8559
Epoch 00001: val_accuracy improved from -inf to 0.89143, saving model to checkpoint/best_val_acc.hdf5
844/844 [==============================] - 2s 3ms/step - loss: 4.9038 - accuracy: 0.8562 - val_loss: 0.4583 - val_accuracy: 0.8914
Epoch 2/20
840/844 [============================>.] - ETA: 0s - loss: 0.3847 - accuracy: 0.9151
Epoch 00002: val_accuracy improved from 0.89143 to 0.92161, saving model to checkpoint/best_val_acc.hdf5
844/844 [==============================] - 2s 2ms/step - loss: 0.3841 - accuracy: 0.9152 - val_loss: 0.3391 - val_accuracy: 0.9216
Epoch 3/20
835/844 [============================>.] - ETA: 0s - loss: 0.3095 - accuracy: 0.9265
Epoch 00003: val_accuracy did not improve from 0.92161
844/844 [==============================] - 2s 2ms/step - loss: 0.3094 - accuracy: 0.9265 - val_loss: 0.4340 - val_accuracy: 0.9104
Epoch 4/20
833/844 [=

In [10]:
scores = model.predict(x_test)

In [11]:
print('deep learned accuracy')
lab = np.argmax(scores, axis=1)
np.mean(y_test==lab)

deep learned accuracy


0.9588

In [12]:
x_train_, y_train_, x_test_, y_test_, _ = importData(0)

Data-set: fashion-mnist


In [13]:
from sklearn.metrics import roc_auc_score, average_precision_score

print('deep learned auroc')
all_x = np.concatenate([x_test, x_test_], axis=0)
all_y = np.zeros(all_x.shape[0], dtype=int)
all_y[:x_test.shape[0]] = 1
scores = model.predict(all_x)
s = np.max(scores, axis=1)
roc_auc_score(all_y, s)

deep learned auroc


0.75331768

In [14]:
from sklearn.svm import SVC, LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification


clf = make_pipeline(StandardScaler(),
                     SVC(random_state=0, tol=1e-5))
clf.fit(x_train, y_train)
label = clf.predict(x_test)
score = clf.decision_function(x_test)
acc = np.mean(label==y_test)
print('SVM accuracy')

print('accuracy:', acc)

SVM accuracy
accuracy: 0.966


In [15]:
print('svm auroc')

score = clf.decision_function(all_x)
s = np.max(score, axis=1)
roc_auc_score(all_y, s)

svm auroc


0.9913135399999999